In [24]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)


In [2]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [3]:
class_names = [
    # "unlabelled",
    "road",
    "sidewalk",
    "building",
    "wall",
    "fence",
    "pole",
    "traffic_light",
    "traffic_sign",
    "vegetation",
    "terrain",
    "sky",
    "person",
    "rider",
    "car",
    "truck",
    "bus",
    "train",
    "motorcycle",
    "bicycle",
]

high_conf_obj_weight = 1.0
static_obj_weight = 0.9
maybe_static_obj_weight = 0.7
noisy_obj_weight = 0.2
dynamic_obj_weight = 0.3

class_weights = [
    static_obj_weight,      # "road"
    high_conf_obj_weight,   # "sidewalk",
    high_conf_obj_weight,   # "building",
    high_conf_obj_weight,   # "wall",
    high_conf_obj_weight,   # "fence",
    high_conf_obj_weight,   # "pole",
    high_conf_obj_weight,   # "traffic_light",
    high_conf_obj_weight,   # "traffic_sign",
    noisy_obj_weight,       # "vegetation",
    noisy_obj_weight,       # "terrain",
    noisy_obj_weight,       # "sky",
    dynamic_obj_weight,     # "person",
    dynamic_obj_weight,     # "rider",
    dynamic_obj_weight,     # "car",
    maybe_static_obj_weight,# "truck",
    maybe_static_obj_weight,# "bus",
    maybe_static_obj_weight,# "train",
    dynamic_obj_weight,     # "motorcycle",
    dynamic_obj_weight,     # "bicycle"
]

In [ ]:
kitti_seg_dir = "/home/moloy/projects/localization/datasets/kitti/sequences/"
output_root_dir = "/home/moloy/projects/localization/kphm-vo/dataset/kitti_odom/related/semantic_heatmap/"
shift_heatmap_dir = "/home/moloy/projects/localization/kphm-vo/dataset/kitti_odom/related/kpheatmap/"
img_folder = "image_3"

In [6]:
for sq_no in range(11):
    working_dir = os.path.join(kitti_seg_dir, 
                                str(sq_no).zfill(2), 
                                img_folder + "_segmented")

    print("Processing sq: ", sq_no)

    all_img_files = os.listdir(working_dir)
    kernel = np.ones((5,5),np.float32)/25

    output_dir = os.path.join(output_root_dir, str(sq_no).zfill(2), img_folder)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    for img_file in all_img_files:
        if not img_file.endswith('.npy'):
            continue

        seg_mask = np.load(os.path.join(working_dir, img_file)).astype(np.float32)
        weighted_seg_mask = np.zeros_like(seg_mask)
        hm_file = str(int(img_file.split('.')[0])).zfill(10) + '.png'
        
        kphm_file = os.path.join(   shift_heatmap_dir, 
                                    str(sq_no).zfill(2), 
                                    img_folder, 
                                    hm_file)

        kphm = np.array(Image.open(kphm_file))

        for i in range(len(class_weights)):
            weighted_seg_mask[seg_mask == i] = class_weights[i]

        resized_seg_mask = cv2.resize(weighted_seg_mask, 
                                    (kphm.shape[1], kphm.shape[0]), 
                                    interpolation = cv2.INTER_LINEAR)

        seg_mask = cv2.resize(seg_mask, 
                                (kphm.shape[1], kphm.shape[0]), 
                                interpolation = cv2.INTER_LINEAR)
        forced_weighted_seg_mask = np.zeros_like(seg_mask, dtype=np.float32)
        forced_weighted_seg_mask[seg_mask == 5] = 180.0
        forced_weighted_seg_mask[seg_mask == 6] = 180.0
        forced_weighted_seg_mask[seg_mask == 7] = 180.0
        forced_weighted_seg_mask = cv2.filter2D(forced_weighted_seg_mask,-1,kernel)

        # semantic_hm = cv2.filter2D(resized_seg_mask,-1,kernel)
        # semantic_kphm = np.multiply(semantic_hm, kphm)
        # semantic_kphm = (semantic_kphm/np.max(semantic_kphm))*255.0
        semantic_kphm = kphm

        forced_weighted_seg_mask[forced_weighted_seg_mask < semantic_kphm] = 0
        semantic_kphm[forced_weighted_seg_mask > 0] = forced_weighted_seg_mask[forced_weighted_seg_mask > 0]

        # plt.imshow(resized_seg_mask, cmap='gray', interpolation='nearest')
        # plt.figure()
        # plt.imshow(kphm, cmap='gray', interpolation='nearest')
        # plt.figure()
        
        semantic_kphm_file = os.path.join(output_dir, hm_file)
        plt.imsave(semantic_kphm_file, semantic_kphm, cmap='gray')

    #     break
    # break


Processing sq:  0
Processing sq:  1
Processing sq:  2
Processing sq:  3
Processing sq:  4
Processing sq:  5
Processing sq:  6
Processing sq:  7
Processing sq:  8
Processing sq:  9
Processing sq:  10
